In [1]:
from pathlib import Path
import numpy as np
import os
from scipy.io import wavfile

In [2]:
project_root = Path(os.getcwd()).parent
results_dir = project_root / "results"
exp_dir = results_dir / "first_with_sound"

In [3]:
wav_path = Path(exp_dir) / "video.wav"
sr, data = wavfile.read(wav_path)

In [4]:
print("sample rate:", sr)
print("shape:", data.shape)
print("dtype:", data.dtype)

sample rate: 44100
shape: (551251,)
dtype: int16


In [5]:
# Convert to float if it's int16, etc.
if np.issubdtype(data.dtype, np.integer):
    max_val = np.iinfo(data.dtype).max
    fdata = data.astype(np.float32) / max_val
else:
    fdata = data.astype(np.float32)

print("min, max:", fdata.min(), fdata.max())
print("mean abs:", np.mean(np.abs(fdata)))

min, max: 0.0 0.0
mean abs: 0.0


In [6]:
from pathlib import Path
import json

from satisfying_sims.core.recording import SimulationRecording
from satisfying_sims.audio.io import load_sample_bank
from satisfying_sims.audio.engine import AudioEngine
from satisfying_sims.audio.mapping import EventSoundMapper
from satisfying_sims.audio.build_soundtrack import (
    make_default_event_sound_rules,
    build_soundtrack_from_snapshots,
)

def debug_soundtrack(
    rec_path: str | Path,
    samples_dir: str | Path,
    sample_map_json: str | None = None,
    sr: int = 44100,
    tail: float = 0.3,
) -> None:
    rec_path = Path(rec_path)
    samples_dir = Path(samples_dir)

    # 1) Load recording
    if rec_path.suffix == ".xz":
        recording = SimulationRecording.load(rec_path)
    else:
        import pickle
        with rec_path.open("rb") as f:
            recording: SimulationRecording = pickle.load(f)

    events = list(recording.iter_events())
    print(f"num events: {len(events)}")

    # 2) Load samples
    samples = load_sample_bank(samples_dir, target_sr=sr, mono=True)
    print(f"sample bank keys: {list(samples.keys())}")

    # 3) Optional sample_map from JSON
    if sample_map_json is not None:
        sample_names = json.loads(sample_map_json)
        print(f"sample_names (from JSON): {sample_names}")
    else:
        sample_names = None

    # 4) Build rules & mapper
    rules = make_default_event_sound_rules(sample_names=sample_names)
    print("rules sample_name per event type:")
    for k, r in rules.items():
        print(f"  {k}: {r.sample_name}")

    mapper = EventSoundMapper(rules)

    # 5) Build triggers
    triggers = mapper.triggers_from_snapshots(events)
    print(f"num triggers: {len(triggers)}")
    if triggers:
        first = triggers[0]
        print(
            "first trigger:",
            f"t={first.t:.4f}, sample_name={first.sample_name},",
            f"gain={first.gain}, pitch_ratio={first.pitch_ratio}",
        )

        # Check that sample_name actually exists in bank
        missing = {tr.sample_name for tr in triggers if tr.sample_name not in samples}
        print("missing sample_names in bank:", missing)

    # 6) Actually mix once and inspect stats
    engine = AudioEngine(samples, sr=sr)
    from satisfying_sims.audio.build_soundtrack import build_soundtrack_from_snapshots as _mix
    audio = _mix(events, mapper, engine, base_duration=recording.t_end, tail=tail)

    import numpy as np
    print("audio min/max:", float(audio.min()), float(audio.max()))
    print("audio mean abs:", float(np.mean(np.abs(audio))))


In [8]:
debug_soundtrack(
    rec_path=str(project_root / "results" / "first_with_sound" / "recording.pkl.xz"),
    samples_dir=str(project_root / "assets/audio"),
    sample_map_json='{"CollisionEvent": "ice_crack", "BoundaryCollisionEvent": "bloop"}',  # or None
)

num events: 25
sample bank keys: ['bloop', 'ice_crack']
sample_names (from JSON): {'CollisionEvent': 'ice_crack', 'BoundaryCollisionEvent': 'bloop'}
rules sample_name per event type:
  CollisionEvent: ice_crack
  BoundaryCollisionEvent: bloop
num triggers: 4
first trigger: t=5.1650, sample_name=ice_crack, gain=1.0, pitch_ratio=1.1999847582938052
missing sample_names in bank: set()
audio min/max: -0.7727461457252502 0.8146445155143738
audio mean abs: 0.0028940460179001093


/Users/mitchweikert/Documents/random_projects/satisfying_sims/src/satisfying_sims/audio/io.py:44: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, data = wavfile.read(path)
